# Stochastic processes and reproducibility

## Generating random numbers

In [1]:
import agentpy as ap
import numpy as np

To illustrate, let us define a model that generates a list of ten pseudo-random numbers:

In [2]:
class RandomModel(ap.Model):
    
    def setup(self):
        self.random_numbers = [self.random.integers(9) for _ in range(10)]
        print(f"Model {self.p.n} generated the numbers {self.random_numbers}")

Now if we run this model multiple times, we will get a different series of numbers:

In [3]:
for i in range(2):
    parameters = {'steps':0, 'n':i}
    model = RandomModel(parameters)
    results = model.run(display=False)

Model 0 generated the numbers [8, 2, 7, 7, 3, 4, 0, 1, 5, 1]
Model 1 generated the numbers [0, 7, 2, 8, 3, 4, 2, 3, 1, 2]


In [4]:
for i in range(2):
    parameters = {'seed':1, 'steps':0, 'n':i}
    model = RandomModel(parameters)
    model.run(display=False)

Model 0 generated the numbers [4, 4, 6, 8, 0, 1, 7, 8, 2, 2]
Model 1 generated the numbers [4, 4, 6, 8, 0, 1, 7, 8, 2, 2]


## Using multiple generators

In [5]:
class RandomAgent2(ap.Agent):
    
    def setup(self):
        seed = self.model.random.bit_generator.random_raw() # Seed from model
        self.random = np.random.default_rng(seed)  # Create agent generator
        self.random_numbers = [self.random.integers(9) for _ in range(10)]
        print(f"{self} generated the numbers {self.random_numbers}")
        
class RandomModel2(ap.Model):
    
    def setup(self):
        print(f"Model {i}:")
        self.add_agents(2, RandomAgent2)
        print()

for i in range(2):
    parameters = {'seed': 1, 'steps': 0}
    model = RandomModel2(parameters)
    results = model.run(display=False)

Model 0:
RandomAgent2 (Obj 1) generated the numbers [5, 7, 3, 8, 8, 6, 2, 2, 0, 6]
RandomAgent2 (Obj 2) generated the numbers [8, 7, 4, 2, 3, 2, 4, 3, 6, 3]

Model 1:
RandomAgent2 (Obj 1) generated the numbers [5, 7, 3, 8, 8, 6, 2, 2, 0, 6]
RandomAgent2 (Obj 2) generated the numbers [8, 7, 4, 2, 3, 2, 4, 3, 6, 3]



Alternatively, we could also have each agent start from the same seed:

In [6]:
class RandomAgent3(ap.Agent):
    
    def setup(self):
        self.random = np.random.default_rng(self.p.agent_seed)
        self.random_numbers = [self.random.integers(9) for _ in range(10)]
        print(f"{self} generated the numbers {self.random_numbers}")
        
class RandomModel3(ap.Model):
    
    def setup(self):
        print(f"\nModel {i}:")
        self.add_agents(2, RandomAgent3)
        
for i in range(2):
    parameters = {'agent_seed': 1, 'steps':0, 'n':i}
    model = RandomModel3(parameters)
    results = model.run(display=False)


Model 0:
RandomAgent3 (Obj 1) generated the numbers [4, 4, 6, 8, 0, 1, 7, 8, 2, 2]
RandomAgent3 (Obj 2) generated the numbers [4, 4, 6, 8, 0, 1, 7, 8, 2, 2]

Model 1:
RandomAgent3 (Obj 1) generated the numbers [4, 4, 6, 8, 0, 1, 7, 8, 2, 2]
RandomAgent3 (Obj 2) generated the numbers [4, 4, 6, 8, 0, 1, 7, 8, 2, 2]


## Modeling stochastic processes

This section presents some stochastic operations that are often used in agent-based models. To start, we prepare a generic model with ten agents:

In [7]:
model = ap.Model()
agents = model.add_agents(9)
agents

AgentList [9 agents]

If we look at the agent's ids, we see that they have been created in order:

In [8]:
agents.id

AttrList of attribute 'id': [1, 2, 3, 4, 5, 6, 7, 8, 9]

In [9]:
agents.shuffle().id

AttrList of attribute 'id': [1, 3, 5, 8, 9, 4, 6, 7, 2]

In [10]:
agents.random(5).id

AttrList of attribute 'id': [6, 3, 1, 2, 9]

In [11]:
for _ in range(2):
    custom_generator = np.random.default_rng(1)
    print(agents.random(5, custom_generator).id)

AttrList of attribute 'id': [4, 6, 3, 8, 6]
AttrList of attribute 'id': [3, 1, 4, 5, 1]


In [12]:
i_sum = sum(agents.id)
weights = [i/i_sum for i in agents.id]
agents.random(10, weights=weights, replace=True).id

AttrList of attribute 'id': [8, 5, 5, 7, 1, 5, 6, 7, 8, 1]

## Further reading

- Random number generation in Python:  
  https://realpython.com/python-random/
- Random sampling in Numpy:  
  https://numpy.org/devdocs/reference/random/index.html
- Stochasticity in agent-based models:  
  http://www2.econ.iastate.edu/tesfatsi/ace.htm#Stochasticity
- Pseudo-random number generators:  
  https://en.wikipedia.org/wiki/Pseudorandom_number_generator
- What is random:  
  https://www.youtube.com/watch?v=9rIy0xY99a0